In [4]:
import cv2
import numpy as np
import os

In [2]:
# Function to resize an image to a fixed size
def resize_image(image, size):
    return cv2.resize(image, size, interpolation=cv2.INTER_AREA)

# Function to read images and convert them to grayscale
def read_images(filenames):
    """Reads images from files and converts them to grayscale."""
    images = [cv2.imread(file) for file in filenames]
    resized_images = [resize_image(img, (512,512)) for img in images]
    images_grey = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in resized_images]
    return images_grey

# Function to detect features and descriptors using SIFT
def detect_features(images_grey):
    """Detects keypoints and computes descriptors for a list of grayscale images."""
    sift = cv2.SIFT_create()
    keypoints_descriptors = []
    for idx, image in enumerate(images_grey):
        keypoints, descriptor = sift.detectAndCompute(image, None)
        keypoints_descriptors.append((keypoints, descriptor))
        img_with_keypoints = cv2.drawKeypoints(image, keypoints, None)
        cv2.imshow(f"Features Image {idx+1}", img_with_keypoints)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    return keypoints_descriptors

# Function to match descriptors between two images
def match_descriptors(descriptor1, descriptor2, ratio_test=0.7):
    # Create a brute force matcher with Euclidean distance
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    
    # Find the two best matches for each descriptor
    matches = bf.knnMatch(descriptor1, descriptor2, k=2)

    # Apply Lowe's ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < ratio_test * n.distance:
            good_matches.append(m)
    return good_matches


def stitch_images(image_left, image_middle, kp_left, kp_middle, matches):
    points_left = np.float32([kp_left[m.queryIdx].pt for m in matches])
    points_middle = np.float32([kp_middle[m.trainIdx].pt for m in matches])

    H, _ = cv2.findHomography(points_left, points_middle, cv2.RANSAC, 5.0)

    # Find the dimensions of the transformed left picture
    h_left, w_left = image_left.shape
    corners_left = np.float32([[0, 0], [0, h_left], [w_left, h_left], [w_left, 0]]).reshape(-1, 1, 2)
    corners_left_transformed = cv2.perspectiveTransform(corners_left, H)
    [x_min, y_min] = np.int32(corners_left_transformed.min(axis=0).ravel() - 0.5)
    [x_max, y_max] = np.int32(corners_left_transformed.max(axis=0).ravel() + 0.5)
    
    # Adjust the homography to translate the image and prevent cropping
    translation_dist = [-x_min, -y_min]
    H_translation = np.array([[1, 0, translation_dist[0]], [0, 1, translation_dist[1]], [0, 0, 1]])
    H_modified = H_translation.dot(H)

    # Make sure it's large enough to contain both images
    output_width = max(x_max, image_middle.shape[1] - x_min)
    output_height = max(y_max, image_middle.shape[0] - y_min)
    output_img_size = (output_width, output_height)

    # Apply the homography with the calculated output size
    transformed_left = cv2.warpPerspective(image_left, H_modified, output_img_size)

    # Create the combined image with the output size
    combined = np.zeros((output_height, output_width), dtype=np.uint8)

    # Ensure the index is not negative
    x_offset = max(translation_dist[0], 0)
    y_offset = max(translation_dist[1], 0)

    # Place the middle image in its new position in the combined image
    combined[y_offset:y_offset+image_middle.shape[0], x_offset:x_offset+image_middle.shape[1]] = image_middle

    # Create a mask for the transformed image
    mask = transformed_left > 0

    # Overlay the transformed image on the combined image
    combined[mask] = transformed_left[mask]

    return combined

In [5]:
# Input image filenames
H1_filenames = ['input_photos/V1_IZQ.jpeg', 'input_photos/V1_MID.jpeg', 'input_photos/V1_DER.jpeg']
H2_filenames = ['input_photos/V2_IZQ.jpeg', 'input_photos/V2_MID.jpeg', 'input_photos/V2_DER.jpeg']
H3_filenames = ['input_photos/V3_IZQ.jpeg', 'input_photos/V3_MID.jpeg', 'input_photos/V3_DER.jpeg']
V_filenames = ['input_photos/H_IZQ.jpeg', 'input_photos/H_MID.jpeg', 'input_photos/H_DER.jpeg']
all_filenames = [V_filenames, H1_filenames, H2_filenames, H3_filenames]

for filenames in all_filenames:
    # Read and convert the images to grayscale
    images_grey = read_images(filenames)

    # Detect features in the grayscale images
    kps_des = detect_features(images_grey)

    # Match descriptors between the first and second image
    matches12 = match_descriptors(kps_des[0][1], kps_des[1][1])

    # Stitch the first two images
    result_12 = stitch_images(images_grey[0], images_grey[1], kps_des[0][0], kps_des[1][0], matches12)

    # Detect features in the stitched image of the first two
    kp_result_12, desc_result_12 = detect_features([result_12])[0]

    # Detect features in the third image
    kp_image_3, desc_image_3 = detect_features([images_grey[2]])[0]

    # Match descriptors between the stitched image and the third image
    matches_12_3 = match_descriptors(desc_result_12, desc_image_3)

    # Calculate the homography between the stitched image and the third image
    points1 = np.zeros((len(matches_12_3), 2), dtype=np.float32)
    points2 = np.zeros_like(points1)

    for i, match in enumerate(matches_12_3):
        points1[i, :] = kp_result_12[match.queryIdx].pt
        points2[i, :] = kp_image_3[match.trainIdx].pt

    H2, _ = cv2.findHomography(points2, points1, cv2.RANSAC, 8.0)

    # Stitch the previous result with the third image
    height, width = result_12.shape[:2]
    new_width = width + images_grey[2].shape[1]

    result_123 = cv2.warpPerspective(images_grey[2], H2, (new_width, height))
    result_123[0:height, 0:width] = result_12

    # Save and display the result
    base_name = "_".join([os.path.splitext(os.path.basename(filename))[0] for filename in filenames])
    output_filename = f'results/{base_name}_panorama_result.png'
    cv2.imwrite(output_filename, result_123)
    resized_img = cv2.resize(result_123, None,  fx=0.5, fy=0.5)
    cv2.imshow("Panoramic Image", resized_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Return the name of the output file
    output_filename